### Goal :

* Construction d'un fichier json structurant l'ensemble du menu avec :
    * separation en sections (entrées, plats principaux, desserts ...)
    * Identification des plats individuels
    * Extraction des prix
    * Taguer tous les plats selon le regime alimentaire.


In [ ]:
pip install openai

  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 717.5/717.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 32.1 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [openai]12/13 [openai]c]
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install anthropic

Note: you may need to restart the kernel to use updated packages.


Import

In [ ]:
import os
import requests
import json
import re
import os
from dotenv import load_dotenv
import time
from anthropic import Anthropic
from openai import OpenAI
import json
import re



Load environment variables

In [11]:
load_dotenv() 

claude_api_key = os.environ.get('claude_api_key')
gpt_api_key = os.environ.get('gpt_api_key')

Paths

In [12]:
marcello_ocr_result_path = "../data/raw_extracted/Azure_doc_intelligence/result_OCR_marcello.txt"
prima_ocr_result_path = "../data/raw_extracted/Azure_doc_intelligence/result_OCR_prima.txt"
output_dir = '../data/segmented_data'

### Structuration du resultat de l'OCR en utilisant le LLM Claude Haiku

### Functions

In [ ]:
def analyze_menu_openai(ocr_text, prompt,api_key):
    client = OpenAI(api_key=api_key)
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user", "content": ocr_text}
            ],
            temperature=0
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Erreur: {e}")
        return None

In [ ]:

def analyze_menu_claude(ocr_text, prompt,api_key):
    client = Anthropic(api_key=api_key)
    try:
        response = client.messages.create(
            model="claude-3-5-sonnet-20241022",  # ou "claude-3-7-sonnet-20250219" pour le plus récent
            max_tokens=8192,
            temperature=0,
            system=prompt,  # Le prompt système va ici, pas dans les messages
            messages=[
                {"role": "user", "content": ocr_text}
            ]
        )
        return response.content[0].text
    except Exception as e:
        print(f"Erreur: {e}")
        return None

In [16]:
def clean_response(response):
    """
    Version minimaliste pour nettoyer les réponses OpenAI.
    
    Args:
        response (dict ou str): Réponse de l'API OpenAI
        
    Returns:
        dict: Menu en JSON
    """
    # Convertir en dict si c'est une string
    if isinstance(response, str):
        response = json.loads(response)
        return response



Definition du prompt

In [17]:
prompt = """Analyse ce texte OCR de menu et retourne uniquement un JSON valide suivant cette structure:

{{
  "menu": {{
    "sections": [
      {{
        "name": "nom_section",
        "items": [
          {{
            "name": "nom_plat",
            "price": {{"value": 12.50, "currency": null}},
            "description": "description_complète",
            "ingredients": ["ingrédient1", "ingrédient2"],
            "dietary": ["végétarien"]
          }}
        ]
      }}
    ]
  }}
}}

Texte OCR: {ocr_text}

Instructions:
1. Identifie automatiquement les sections (entrées, plats, desserts, pizzas, boissons, etc.)
2. Pour chaque item: nom, prix, description, ingrédients (déduis-les de la description si nécessaire)
3. Prix: utilise uniquement €, $, £, CHF pour currency. Si autre chose ou illisible, mets null

IMPORTANT - Régimes alimentaires (sois très prudent):
- Si tu as un grand doute, laisse dietary vide []
- Règles strictes:
  * "végétarien": AUCUNE viande, poisson, fruits de mer (mais œufs/lait OK)
  * "végétalien": AUCUN produit animal (pas de viande, poisson, œufs, lait, miel, beurre)
  * "sans_gluten": AUCUN blé, orge, seigle, avoine (attention aux sauces, panure)
  * "sans_lactose": AUCUN lait, crème, fromage, beurre, yaourt

ATTENTION - VIANDES (jamais végétarien):
- Jambon, jambon blanc, jambon cru, prosciutto = VIANDE
- Bacon, lardons, pancetta = VIANDE  
- Saucisse, chorizo, pepperoni = VIANDE
- Salami, coppa, bresaola = VIANDE
- Bœuf, porc, agneau, veau = VIANDE
- Poulet, canard, dinde = VIANDE

Exemples:

VÉGÉTARIEN + VÉGÉTALIEN:
- Salade verte simple = ["végétarien", "végétalien"]
- Légumes grillés sans sauce = ["végétarien", "végétalien"] 
- Frites maison = ["végétarien", "végétalien"]
- Soupe de légumes (bouillon végétal) = ["végétarien", "végétalien"]

VÉGÉTARIEN SEULEMENT:
- Pâtes au beurre = ["végétarien"] (beurre = produit laitier)
- Pizza margherita = ["végétarien"] (fromage = produit laitier)
- Omelette = ["végétarien"] (œufs OK pour végétarien)

SANS GLUTEN SEULEMENT:
- Steak grillé nature = ["sans_gluten"] (pas de panure ni sauce)
- Salade de riz = ["sans_gluten"] (riz OK)
- Poisson grillé nature = ["sans_gluten"]

SANS LACTOSE SEULEMENT:
- Pâtes à l'huile d'olive = ["sans_lactose"] (pas de beurre/fromage)
- Viande grillée nature = ["sans_lactose"]

COMBINAISONS:
- Salade de quinoa aux légumes = ["végétarien", "végétalien", "sans_gluten", "sans_lactose"]
- Riz sauté aux légumes = ["végétarien", "végétalien", "sans_gluten", "sans_lactose"]
- Steak frites = ["sans_gluten", "sans_lactose"] (si frites maison)
IMPORTANT : il faut inclure le resultat complet (tous les elements presents dans le texte OCR)
Retourne uniquement le JSON, sans texte additionnel."""

Test on Marcello menu

In [ ]:
with open(marcello_ocr_result_path, "r", encoding="utf-8") as file:
    marcello_ocr_result = file.read()

'Classiques\nVERDURA\n12,90\ntomate\', aubergines grillées, poivrons grillés, champignons\nde Paris, oignons rouges, olives Leccine, huile de basilic, mozzarella\nfdl\nMIELINA\n12.90\ncrème, chèvre, miel, mozzarella fdl\'\nTARTUFO\n14.90\ncrème de truffes et champignons, mozzarella fdl\'\n(après cuisson, roquette, bresaola, copeaux de parmesan AOC)\nMARGHERITA\n9.90\ntomate\', olives Leccine, huile de basilic, mozzarella fdl\'\nREGINA\n11,90\ntomate\', jambon blanc italien, champignons de Paris,\nolives Leccine, huile de basilic, mozzarella fdl\'\nPARMA\n14,90\ntomate @ huile de basilic, mozzarella fdl\' (après cuisson,\nroquette, jambon de Parme, burrata, tomates cerises,\ncopeaux de parmesan AOC, crème de balsamique)\nQUATTRO FORMAGGI\n12,00\ntomate\', taleggio AOC, gorgonzola DOP,\nparmigiano AOC, mozzarella fdl\'\nCALZONE CLASSICA\n12,90\ntomate\', jambon blanc italien, oeuf, champignons de Paris,\nhuile de basilic, mozzarella fdl"\nMARCELLO\n14.90\ncrème, gorgonzola DOP, coppa, hu

In [ ]:
# result_marcello = analyze_menu_openai(marcello_ocr_result, prompt,openai_api_key)

In [ ]:
result_marcello = analyze_menu_claude(marcello_ocr_result, prompt,claude_api_key)

In [ ]:
cleaned_result_marcello = clean_response(result_marcello)

In [ ]:
# Enregistrement du resultat dans un fichier
with open(output_dir + "/result_LLM_sonnet_marcello", "w") as file: 
        json.dump(cleaned_result_marcello, file, indent=4, ensure_ascii=False)

Test on Prima lova menu

In [286]:
with open(prima_ocr_result_path, "r", encoding="utf-8") as file:
    prima_ocr_result = file.read()

In [ ]:
# result_prima = analyze_menu_openai(prima_ocr_result, prompt,openai_api_key)

In [ ]:
result_prima = analyze_menu_claude(prima_ocr_result, prompt,claude_api_key)

In [ ]:
cleaned_result_prima = clean_response(result_prima)

In [ ]:
# Enregistrement du resultat dans un fichier
with open(output_dir + "/result_LLM_sonnet_prima", "w") as file: 
        json.dump(cleaned_result_prima, file, indent=4, ensure_ascii=False)

### Conclusion

* Le modele Haiku de Claude ne peut pas gerer les gros menu a cause de la limite de tokens
* gpt-4o-mini n'est pas limité par la limite de token dans notre cas d'usage mais le temps de traitement dure plus de 2 min pour les gros menu et 22 secondes pour les petit menu (2 fois plus lent que claude haiku)
* resultats legerement superieur pour claude haiku
* Le modele presentant les meilleurs ressultats pour le moment c'est le modele sonnet de Claude (plus rapide, plus performant, peut supporter la limite de toekn pour notre cas d'usage)
* Point d'attention : bien que sonnet presente des bons resultats, les modeles sont trop lent. Trouver une solution.
* Piste a explorer : 
*   Tester le streaming, refaire le prompt pour que le modele nous envoie un format jsonL pour pas avoir de probleme de format de la reponse. 
